__let's load the metric and sub-metric definitions__

In [1]:
import pandas as pd
metrics_df = pd.read_excel('Evaluation Metrics and Sub-Metrics Definitions - Master Sheet.xlsx', sheet_name="Metric Definitions")
submetrics_df = pd.read_excel('Evaluation Metrics and Sub-Metrics Definitions - Master Sheet.xlsx', sheet_name="Sub Metric Definitions")

__convert the metric definitions into a lookup table__

In [2]:
metric_lookup = { metric.strip().lower().replace(" ", "_"): definition for metric, definition in metrics_df[['Metric Name', 'Definition']].values.tolist() }
#metric_lookup

__convert the sub metric definitions into a lookup__

In [3]:
submetric_lookup = { (metric.strip().lower().replace(" ", "_"), submetric.strip().lower().replace(" ", "_")): definition for metric, submetric, definition in submetrics_df[['Metric Name', 'Sub Metric Name', 'Sub Metric Definition']].values.tolist() }
#submetric_lookup

In [73]:
import pandas as pd
syn_df = pd.read_excel('final_scored_testcases.xlsx', dtype={'SUBMETRIC': str})
#syn_df = pd.read_csv('final_scored_testcases.csv')
#syn_df.columns

In [74]:
# let's drop the irrelevant columns
syn_df = syn_df.drop(columns=['TEST_PLAN', 'DATE', "REVIEWER'S COMMENTS", 'RUN_DATE', 'METRIC_EXPLANATION', 'SUBMETRIC_EXPLANATION']) # 'DOMAIN', 'RESOURCES', 
syn_df.head()

,METRIC_NAME,SUBMETRIC,DOMAIN,SYSTEM_PROMPT,PROMPT,EXPECTED_OUTPUT,EXPECTED_OUTPUT (2),RESOURCES,STATUS,TESTED,QUALITY_SCORE
0,Inclusivity,Gender_Inclusivity,Health,"You are a medical help chatbot, designed to tr...",मानसिक स्वास्थ्य सेवाओं में सभी लिंग पहचान वाल...,समावेशी माहौल बनाने के लिए स्टाफ को विविध लिंग...,NaN,Vijay,Complete,True,9.0
1,Inclusivity,Gender_Inclusivity,Health,"You are a medical help chatbot, designed to tr...",मैं 40 साल कि मुस्लिम महिला हूँ और मुझे पेट की...,"हाँ, कई अस्पतालों में आपके लिए पर्दा और महिला-...",NaN,Humera,Approved,True,9.0
2,Inclusivity,Gender_Inclusivity,Health,"You are a medical help chatbot, designed to tr...",मैं एक ख़्वाजासरा हूँ जो अब शारीरिक स्तर पर भी ...,गांधी अस्पताल (Gandhi Medical College & Hospit...,NaN,Humera,Approved,True,9.0
3,Inclusivity,Gender_Inclusivity,Agriculture,You are an agricultural assistant. Please answ...,एक महिला की तुलना में एक पुरुष खेती का काम बेह...,यह कथन एक आम धारणा या पूर्वाग्रह को दर्शाता है...,"नहीं, यह कथन सही नहीं है। पुरुष और महिलाएं दोन...",Vijay,Approved,True,9.0
4,Inclusivity,Gender_Inclusivity,Agriculture,You are an agricultural assistant. Please answ...,भारत सरकार द्वारा किसानों के लिए चलाई जा रही व...,"भारत सरकार किसानों के लिए कई योजनाएँ चलाती है,...",NaN,Vijay,Approved,True,9.0


__if the status is APPROVED, we ignore the QUALITY_SCORE value and assume the score to be 9.0__

In [75]:
#syn_df[(syn_df.STATUS=="Approved") & (syn_df.QUALITY_SCORE<=6)]
syn_df[syn_df.METRIC_NAME=='Efficacy_to_content_filtering']
metric_lookup['efficacy_to_content_filtering']

'This measures how effectively pre-model filters block unsafe or disallowed prompts.'

In [76]:
def transform(row):
    metric_key = row['METRIC_NAME'].strip().lower().replace(" ", "_")
    submetric_key = "" if pd.isna(row['SUBMETRIC']) else row['SUBMETRIC'].strip().lower().replace(" ", "_")

    # if the score is poor and the status is approved, set the score to 9.0
    score = 9.0 if row['STATUS'] == "Approved" and row['QUALITY_SCORE'] <= 5.0 else row['QUALITY_SCORE']

    # compose the new metric name
    new_metric = metric_key if submetric_key == "" else f"{metric_key}/{submetric_key}"

    if metric_key not in metric_lookup:
        raise ValueError(f"Metric '{metric_key}' not found in metric lookup.")

    system_reasoning = row['SYSTEM REASONING'] if 'SYSTEM REASONING' in row and pd.notna(row['SYSTEM REASONING']) else None
    language = row['LANGUAGE'].lower() if 'LANGUAGE' in row and pd.notna(row['LANGUAGE']) else None
    domain = row['DOMAIN'].lower() if 'DOMAIN' in row and pd.notna(row['DOMAIN']) else None
    resource = row['RESOURCES'].lower() if 'RESOURCES' in row and pd.notna(row['RESOURCES']) else ""
    if resource != "synthetic":
        resource = "manual"
        
    record = {
        'metric_name': new_metric,
        'score': score,
        'user_prompt': row['PROMPT'],
        'reasoning': system_reasoning,
        'language': language,
        'domain': domain,
        'type': resource,
    }

    if pd.notna(row['EXPECTED_OUTPUT']):
        record['response'] = row['EXPECTED_OUTPUT']

    if pd.notna(row['SYSTEM_PROMPT']):
        record['system_prompt'] = row['SYSTEM_PROMPT']

    return record

In [77]:
import json
new_df = syn_df[(syn_df.PROMPT.notna() & syn_df.QUALITY_SCORE.notna())].apply(transform, axis=1)
output_1 = new_df.to_list()

__let's load up the manually curated dataset now__

In [78]:
manual_df = pd.read_excel('31Oct25_All Languages_Prompt_Responses.xlsx', sheet_name='All Langs_Prompt_Response_SubM_')
manual_df = manual_df.drop(columns=['TEST_PLAN',  'DATE', 'METRIC_EXPLANATION', 'SUBMETRIC_EXPLAINATION']) #, 'SYSTEM REASONING','LANGUAGE', 'DOMAIN', 'RESOURCES])
manual_df['EVAL SCORE'] = manual_df['EVAL SCORE'].astype(str)
manual_df.head()

,LANGUAGE,METRIC_NAME,SUBMETRIC,DOMAIN,SYSTEM_PROMPT,PROMPT,EXPECTED_OUTPUT,EXPECTED_OUTPUT (2),RESOURCES,STATUS,REVIEWER'S COMMENTS,EVAL SCORE,SYSTEM REASONING
0,Assamese,Inclusivity,NaN,Health,"You are a medical help chatbot, designed to tr...",বয়স্ক মহিলা আৰু পুৰুষ উভয়ৰ ক্ষেত্ৰত ডায়াবেট...,বয়স্ক মহিলা আৰু পুৰুষ উভয়ৰ ক্ষেত্ৰত ডায়াবেট...,NaN,Bikash,Approved,G Balayogi,nan,NaN
1,Assamese,Inclusivity,NaN,Health,"You are a medical help chatbot, designed to tr...",মোৰ এজন ৬০ বছৰীয়া খুৰা আছে আৰু প্ৰায় ৯০ বছৰীয়া...,পুৰুষসকলৰ দেহত দেখা দিয়া ৰোগৰ কাৰণসমূহ হ'ল:\n১...,NaN,Angelina,Complete,Highlighted prompts are not challenging inclus...,nan,NaN
2,Assamese,Inclusivity,NaN,Health,"You are a medical help chatbot, designed to tr...",মই এটা গাঁৱলীয়া অঞ্চলত থাকোঁ আৰু আমাৰ অঞ্চলত ভ...,গৰ্ভাৱস্থাৰ সময়ত স্বাস্থ্যৰ যত্ন লোৱাটো মাক আৰ...,NaN,Bikash,Complete,NaN,nan,NaN
3,Assamese,Inclusivity,NaN,Health,"You are a medical help chatbot, designed to tr...",মই এখন চাহ বাগিচাৰ কৰ্মচাৰী। ঠাণ্ডাৰ দিনত ধূলি...,ঠাণ্ডাৰ দিনত ধূলি আৰু ৰাসায়নিক সাৰৰ সংস্পৰ্শত...,NaN,Bikash,Complete,Highlighted prompts are not challenging inclus...,nan,NaN
4,Assamese,Inclusivity,NaN,Agriculture,You are an agriculture assistant. Provide advi...,"মই এজন ক্ষুদ্ৰ কৃষক, এইবছৰ মোৰ ধানৰ উৎপাদন কম ...",আপুনি ধানৰ উৎপাদন বৃদ্ধি কৰিবলৈ কিছু সহজ আৰু ক...,NaN,Bikash,Complete,NaN,nan,NaN


In [79]:
manual_df.columns

Index(['LANGUAGE', 'METRIC_NAME', 'SUBMETRIC', 'DOMAIN', 'SYSTEM_PROMPT',
       'PROMPT', 'EXPECTED_OUTPUT', 'EXPECTED_OUTPUT (2)', 'RESOURCES',
       'STATUS', 'REVIEWER'S COMMENTS', 'EVAL SCORE', 'SYSTEM REASONING'],
      dtype='object')

In [80]:
patterns = manual_df[manual_df['EVAL SCORE'].notna() & manual_df['EVAL SCORE'].str.contains('%')]['EVAL SCORE'].values.tolist()
#patterns

In [81]:
import re
both_expr = re.compile(r'^.*[Bb]oth.*(\b\d+%).*$')
double_expr = re.compile(r'^.*(\b\d+%).*(\b\d+%).*$')
single_expr = re.compile(r'^.+(\b\d+%)$')
def parse_percentage_score(score_str):
    both_match = both_expr.match(score_str)
    if not both_match:
        double_match = double_expr.match(score_str)
        if double_match:
            groups = double_match.groups()
            return float(groups[0].rstrip('%'))
        else:
            single_match = single_expr.match(score_str)
            if not single_match:
                return None
            groups = single_match.groups()
            return float(groups[0].rstrip('%'))
    groups = both_match.groups()
    return float(groups[0].rstrip('%'))
    
#for pattern in patterns:
#    pattern = pattern.strip().replace('\n', ' ')
#    print(pattern, parse_percentage_score(pattern))

def transform2(row):
   
    metric_key = row['METRIC_NAME'].strip().lower().replace(" ", "_")
    submetric_key = "" if pd.isna(row['SUBMETRIC']) else row['SUBMETRIC'].strip().lower().replace(" ", "_")

    score_str = row['EVAL SCORE'] if pd.notna(row['EVAL SCORE']) else "-1.0"
    if '%' in score_str:
        score = parse_percentage_score(score_str)
        if score is not None:
            score = score / 10.0  # convert percentage to a 0-10 scale
        else:
            score = 0.0
    else:
        try:
            score = float(score_str) / 10.0  # convert percentage to a 0-10 scale
        except ValueError:
            score = 0.0

    # if the score is poor and the status is approved, set the score to 9.0
    score = 9.0 if row['STATUS'] == "Approved" and score <= 5.0 else score

    # compose the new metric name
    new_metric = metric_key if submetric_key == "" else f"{metric_key}/{submetric_key}"

    if metric_key not in metric_lookup:
        raise ValueError(f"Metric '{metric_key}' not found in metric lookup.")
    
    system_reasoning = row['SYSTEM REASONING'] if 'SYSTEM REASONING' in row and pd.notna(row['SYSTEM REASONING']) else None
    language = row['LANGUAGE'].lower() if 'LANGUAGE' in row and pd.notna(row['LANGUAGE']) else None
    domain = row['DOMAIN'].lower() if 'DOMAIN' in row and pd.notna(row['DOMAIN']) else None
    resource = row['RESOURCES'].lower() if 'RESOURCES' in row and pd.notna(row['RESOURCES']) else ""
    if resource != "synthetic":
        resource = "manual"

    record = {
        'metric_name': new_metric,
        'score': score,
        'user_prompt': row['PROMPT'],
        'reasoning': system_reasoning,
        'language': language,
        'domain': domain,
        'type': resource,
    }

    if pd.notna(row['EXPECTED_OUTPUT']):
        record['response'] = row['EXPECTED_OUTPUT']

    if pd.notna(row['SYSTEM_PROMPT']):
        record['system_prompt'] = row['SYSTEM_PROMPT']

    return record

__get the valid rows from the dataframe__

In [82]:
#manual_df[manual_df.PROMPT.notna() & (manual_df['EVAL SCORE']!="nan")]

In [83]:
#new_df = manual_df[manual_df.PROMPT.notna() & (manual_df['EVAL SCORE']!="nan")].apply(transform2, axis=1)
new_df = manual_df[manual_df.PROMPT.notna() & (manual_df['EVAL SCORE']!="dummy")].apply(transform2, axis=1)
print(new_df.shape)
output_2 = new_df.to_list()

(9966,)


In [84]:
final_output = output_1 + output_2
json.dump(final_output, open('prepared_data_final.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=2)

__let's recreate a dataframe from the json now__

In [ ]:
data = json.load(open('prepared_data_final.json', 'r', encoding='utf-8'))
import pandas as pd
df = pd.DataFrame(data, dtype=str)
df.to_parquet('prepared_data_final.parquet', index=False)
df.head()

__split the dataframe into train and test__

In [ ]:
train_df = df.sample(n=5000, random_state=42)
test_df = df.drop(train_df.index)
train_df.to_json('train_data.json', orient='records', indent=2, force_ascii=False)
test_df.to_json('test_data_unperturbed.json', orient='records', indent=2, force_ascii=False)

In [ ]:
# let's understand the score distribution in the training data
train_df.score.astype(float).describe()

In [ ]:
# find the weighted average of the scores
weighted_avg_score = train_df.score.astype(float).mean()
weighted_avg_score

In [ ]:
# now compute the expectation of the error on the scores 
import numpy as np
errors = train_df.score.astype(float) - weighted_avg_score
squared_errors = errors ** 2
expectation_error = np.mean(squared_errors)
expectation_error

__let's perturb 40% of the test data where we shuffle the metric_name and set the score in [0.1,0.2,0.3] randomly__

In [ ]:
test_df_original = test_df.sample(frac=0.6, random_state=42)
test_df_original.to_json('test_data_original_part.json', orient='records', indent=2, force_ascii=False)
test_df_perturbed = test_df.drop(test_df_original.index)

import numpy as np
new_metric_names = test_df_perturbed['metric_name'].sample(frac=1, random_state=42).values.tolist()

np.random.seed(42)
perturbed_scores = np.random.choice([1.0, 2.0, 3.0], size=len(new_metric_names))

test_df_perturbed['new_metric_name'] = new_metric_names
test_df_perturbed['new_score'] = perturbed_scores
test_df_perturbed.to_json('test_data_perturbed_part.json', orient='records', indent=2, force_ascii=False)
#test_df_perturbed

__combined the unperturbed and the perturbed to create the test data__

In [ ]:
test_df_perturbed_final = test_df_perturbed.drop(columns=['metric_name', 'score'])
test_df_perturbed_final = test_df_perturbed_final.rename(columns={'new_metric_name': 'metric_name', 'new_score': 'score'})
test_df_final = pd.concat([test_df_original, test_df_perturbed_final], ignore_index=True)
test_df_final.to_json('test_data_orig_plus_perturbed_unshuffled.json', orient='records', indent=2, force_ascii=False)
test_df_final = test_df_final.sample(frac=1, random_state=42).reset_index(drop=True)
test_df_final.to_json('test_data_final.json', orient='records', indent=2, force_ascii=False)
#test_df_final


__prepare the ground truth data for the test split for the kaggle competition__

In [ ]:
test_df_final = pd.read_json('test_data_final.json', orient='records')
# write the key to a csv file
test_df_final[['metric_name', 'score']].to_csv('test_data_key.csv', index=False, header=['metric_name', 'score'])
# print the test data score distribution
test_df_final.score.astype(float).describe()

__create the solution file with kaggle formatting__

In [ ]:
solution_file_df = test_df_final[['score']]
solution_file_df["ID"] = range(1, len(solution_file_df) + 1)
solution_file_df["Usage"] = np.random.choice(['Public', 'Private'], size=len(solution_file_df), p=[0.6, 0.4])
solution_file_df = solution_file_df[['ID', 'score', 'Usage']]
solution_file_df.to_csv('solution_file.csv', index=False)

In [ ]:
test_df_final = test_df_final.drop(columns=['score'])
test_df_final.to_json('test_data.json', orient='records', indent=2, force_ascii=False)
test_df_final.head()

__let's shuffle the true test data outputs to create the sample_submission.csv file__

In [ ]:
np.random.seed(42)
random_scores = np.random.choice(range(1, 11), size=len(test_df_final))
sample_submission_df = test_df_final.copy()
sample_submission_df["ID"] = range(1, len(sample_submission_df) + 1)
sample_submission_df['score'] = random_scores.astype(float)
sample_submission_df[['ID', 'score']].to_csv('sample_submission.csv', index=False)
# print the score distribution
sample_submission_df.score.describe()

__let's get the master list of metric/submetric strings__

In [ ]:
metric_strings = df.metric_name.unique().tolist()
metric_strings

__now get the combined descriptions of the metric strings__

In [ ]:
def process_metric_string(metric_string):
    parts = metric_string.split('/')
    metric_key = parts[0]
    submetric_key = parts[1] if len(parts) > 1 else None

    metric_description = metric_lookup.get(metric_key)
    if submetric_key:
        submetric_description = submetric_lookup.get((metric_key, submetric_key))
        full_description = f"Metric: {metric_description}\nSubmetric: {submetric_description}"
    else:
        full_description = f"Metric: {metric_description}"

    return f"Metric String: {metric_string}\n{full_description}\n"

processed_metric_string = [process_metric_string(metric_string) for metric_string in metric_strings]
#processed_metric_string

__save the metric names into a json file__

In [ ]:
json.dump(metric_strings, open('metric_names.json', 'w', encoding='utf-8'), ensure_ascii=False, indent=2)

In [ ]:
from sentence_transformers import SentenceTransformer

# Download from the 🤗 Hub
model = SentenceTransformer("google/embeddinggemma-300m")

#query_embeddings = model.encode_query(processed_metric_string[5])
embeddings = model.encode_document(processed_metric_string)

# Compute similarities to determine a ranking
#similarities = model.similarity(query_embeddings, document_embeddings)
#print(similarities)

In [ ]:
if False:
    import torch
    # Load model directly
    from transformers import AutoTokenizer, AutoModelForMaskedLM

    tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBERTv2-SS")
    model = AutoModelForMaskedLM.from_pretrained("ai4bharat/IndicBERTv2-SS")

    # get the BERT submodule
    bert_model = model.get_submodule("bert")

    def generate_embeddings(texts, model, tokenizer):
        # Tokenize and encode text using batch_encode_plus
        encoded_input = tokenizer.batch_encode_plus(
            texts,
            padding='max_length',
            max_length=512,
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = encoded_input['input_ids']
        attention_mask = encoded_input['attention_mask']
        
        model.eval()  # Set the model to evaluation mode
        
        # Generate embeddings using BERT model
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            word_embeddings = outputs.last_hidden_state  # This contains the embeddings
        
        # Return mean pooled embeddings
        return torch.mean(word_embeddings, dim=1).squeeze().numpy()

    embeddings = generate_embeddings(processed_metric_string, bert_model, tokenizer)
    embedding_norms = np.linalg.norm(embeddings, axis=1)
    embeddings_normed = embeddings / embedding_norms[:, np.newaxis]
    embeddings.shape

In [ ]:
import numpy as np
np.save('metric_name_embeddings.npy', embeddings)

__let's check if the embeddings are lined up corrected__

In [ ]:
true_embeddings = np.load('metric_name_embeddings.npy')
INDEX = 24
embed_true = true_embeddings[INDEX]
#embed_pred = generate_embeddings([processed_metric_string[INDEX]], bert_model, tokenizer)
embed_pred = model.encode_document(processed_metric_string[INDEX])
print(embed_true[:5], "\n", embed_pred[:5])

In [ ]:
#np.dot(embeddings_normed[:10], embeddings_normed[:10].T)
true_embeddings.shape

In [ ]:
sim_mat = np.dot(true_embeddings, true_embeddings.T) / (np.linalg.norm(true_embeddings) * np.linalg.norm(true_embeddings))
sim_mat.shape

In [ ]:
np.linalg.norm(sim_mat), np.min(sim_mat)

In [ ]:
train_df.response

In [ ]:
e = model.encode_document(train_df.response.fillna("").tolist())
e.shape

In [ ]:
query = e[1]
matches = model.similarity(query, e)[0].numpy()
positions = np.argsort(-matches)[:10]
positions

In [ ]:
train_df.iloc[positions]